In [35]:
import pandas as pd  
import numpy as np

In [4]:
df_bridges = pd.read_excel("../data/raw/BMMS_overview.xlsx")
print(df_bridges.columns)
print(df_bridges.head(3))
print("rows:", len(df_bridges))

Index(['road', 'km', 'type', 'LRPName', 'name', 'length', 'condition',
       'structureNr', 'roadName', 'chainage', 'width', 'constructionYear',
       'spans', 'zone', 'circle', 'division', 'sub-division', 'lat', 'lon',
       'EstimatedLoc'],
      dtype='object')
  road     km              type  LRPName               name  length condition  \
0   N1  1.800       Box Culvert  LRP001a                  .   11.30         A   
1   N1  4.925       Box Culvert  LRP004b                  .    6.60         A   
2   N1  8.976  PC Girder Bridge  LRP008b  Kanch pur Bridge.  394.23         A   

   structureNr                                           roadName chainage  \
0       117861  Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...      1.8   
1       117862  Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...    4.925   
2       119889  Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...    8.976   

   width  constructionYear  spans   zone circle     division   sub-division  \
0   19.5       

In [5]:
df_bridges.head()

,road,km,type,LRPName,name,length,condition,structureNr,roadName,chainage,width,constructionYear,spans,zone,circle,division,sub-division,lat,lon,EstimatedLoc
0,N1,1.800,Box Culvert,LRP001a,.,11.30,A,117861,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,1.8,19.5,2005.0,2.0,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.702889,90.450389,bcs1
1,N1,4.925,Box Culvert,LRP004b,.,6.60,A,117862,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,4.925,35.4,2006.0,1.0,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.693611,90.478833,bcs1
2,N1,8.976,PC Girder Bridge,LRP008b,Kanch pur Bridge.,394.23,A,119889,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,8.976,NaN,NaN,NaN,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.704583,90.518833,road_precise
3,N1,10.880,Box Culvert,LRP010b,NOYAPARA CULVERT,6.30,A,112531,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,10.88,12.2,1992.0,2.0,Dhaka,Dhaka,Narayanganj,Vitikandi,23.699833,90.530722,bcs1
4,N1,10.897,Box Culvert,LRP010c,ADUPUR CULVERT,6.30,A,112532,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,10.897,12.2,1984.0,2.0,Dhaka,Dhaka,Narayanganj,Vitikandi,23.699667,90.530722,bcs1


In [6]:
df[["km","lat","lon"]].describe()

,km,lat,lon
count,21406.000000,21313.000000,21313.000000
mean,41.217405,23.828505,90.227328
std,78.810955,2.247895,3.364169
min,0.000000,0.000000,0.000000
25%,6.207250,22.942527,89.377806
50%,15.147000,23.823743,90.289194
75%,35.012250,24.720939,91.288417
max,522.718000,91.544194,93.298416


In [ ]:
#dit is eig cluster 1: de bridges met sws verkeerde waardes 
# 1) Missing coords (NAN values)
mask_missing = df_bridges["lat"].isna() | df_bridges["lon"].isna() 
#--> oplossing: kijken in de andere data sets of het daar wel in staat? je weet wel weg waar op zou staan

# 2) Zero coords (0 or 0.0)
mask_zero = (df_bridges["lat"] == 0) | (df_bridges["lon"] == 0) 


# 3) Invalid range coords
mask_invalid = (df["lat"] < -90) | (df["lat"] > 90) | (df["lon"] < -180) | (df["lon"] > 180)

print("Missing:", mask_missing.sum())
print("Zero:", mask_zero.sum())
print("Invalid:", mask_invalid.sum())

# (optioneel) laat voorbeelden zien
df.loc[mask_invalid, ["road", "km", "lat", "lon"]].head(10) #dit zijn maar kleine fouten net boven 91 graden niet te veel aandacht aan? 
df.loc[mask_zero, ['']

Missing: 94
Zero: 17
Invalid: 15


,road,km,lat,lon
2864,R241,29.840,91.543889,24.805111
2865,R241,31.212,91.538722,24.816556
2866,R241,32.713,91.530556,24.828056
2867,R241,33.126,91.527917,24.830944
2868,R241,34.978,91.518194,24.844972
2869,R241,36.164,91.514083,24.855083
16183,R241,26.292,91.544194,24.773694
16185,R241,28.857,91.542083,24.796694
17247,R241,28.211,91.541778,24.790944
17248,R241,32.128,91.534361,24.823833


NameError: name 'df_roads_clean' is not defined

In [7]:
LAT_MIN, LAT_MAX = 20.5, 26.8
LON_MIN, LON_MAX = 88.0, 92.8

out_country = df[
    (df["lat"].notna()) &
    (df["lon"].notna()) &
    (
        (df["lat"] < LAT_MIN) |
        (df["lat"] > LAT_MAX) |
        (df["lon"] < LON_MIN) |
        (df["lon"] > LON_MAX)
    )
]

print("Outside Bangladesh:", len(out_country)) 


Outside Bangladesh: 51


In [22]:
#cleaned df = clean csv 
df_cleaned_roads = pd.read_csv("../data/raw/clean.csv") 
df_cleaned_roads.head()   
df_bridges.columns

#match them on road and LRP name

Index(['road', 'km', 'type', 'LRPName', 'name', 'length', 'condition',
       'structureNr', 'roadName', 'chainage', 'width', 'constructionYear',
       'spans', 'zone', 'circle', 'division', 'sub-division', 'lat', 'lon',
       'EstimatedLoc'],
      dtype='object')

In [21]:
df_cleaned_roads.columns

Index(['Unnamed: 0', 'road', 'chainage', 'lrp', 'lat', 'lon', 'gap', 'type',
       'name'],
      dtype='object')

In [23]:
def norm(s):
    return s.astype(str).str.strip().str.upper()

# Normalised keys
df_roads = df_cleaned_roads.copy()
df_br = df_bridges.copy()

df_roads["road_norm"] = norm(df_roads["road"])
df_roads["lrp_norm"]  = norm(df_roads["lrp"])

df_br["road_norm"] = norm(df_br["road"])
df_br["lrp_norm"]  = norm(df_br["LRPName"])

In [32]:
#df_br.describe has: 21407 rows

In [33]:
#df_roads.describe has: #52210 rows

In [ ]:
#cluster 2 bridges die niet in road file zitten 
#bridges missing in roads; #dus bridge zegt dat het op bepaalde road ligt maar code komt niet overeen met een 
#LRP code op die road 
bridge_keys = df_br[["road_norm","lrp_norm"]]
road_keys   = df_roads[["road_norm","lrp_norm"]]

bridges_missing_in_roads = bridge_keys.merge(
    road_keys, on=["road_norm","lrp_norm"], how="left", indicator=True
).query("_merge == 'left_only'").drop(columns="_merge")

print("Bridges (BMMS) missing in roads:", len(bridges_missing_in_roads))
bridges_missing_in_roads.head(20)

Bridges (BMMS) missing in roads: 6552


,road_norm,lrp_norm
0,N1,LRP001A
1,N1,LRP004B
11,N1,LRP013B
12,N1,LRP014A
15,N1,LRP018C
17,N1,LRP021C
18,N1,LRP021D
19,N1,LRP021C
21,N1,LRP023D
22,N1,LRP023C


In [ ]:
#Wat ik nu nodig heb: outlier file (groupering dus wat Yoran geinterpoleerd heeft) 
#--> die interpolatie dan aanpssen de lon en lat? in bridge file? Maar wat als ze wel goed stonden in bridge file?